In [1]:
import pandas as pd

file_names = [
    '../data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv', 
    '../data/Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv', 
    '../data/Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014.csv', 
    '../data/Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv', 
    '../data/Resale flat prices based on registration date from Jan-2017 onwards.csv'
]

for file in file_names:
    print("="*80)
    print(f"Reviewing file: {file}")
    try:
        df = pd.read_csv(file)
        print("Shape:", df.shape)
        
        print("\nColumns:")
        for i, col in enumerate(df.columns, start=1):
            print(f"{i}. {col} (dtype: {df[col].dtype})")
        
        print("\nFirst 5 rows:")
        print(df.head())
        
        print("\nDistinct value counts per column:")
        for col in df.columns:
            unique_count = df[col].nunique()
            print(f"{col}: {unique_count} unique values")
    except Exception as e:
        print("Error reading file:", file)
        print(e)
    print("\n")


/Users/bkuzeyardic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Reviewing file: ../data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv
Shape: (287196, 10)

Columns:
1. month (dtype: object)
2. town (dtype: object)
3. flat_type (dtype: object)
4. block (dtype: object)
5. street_name (dtype: object)
6. storey_range (dtype: object)
7. floor_area_sqm (dtype: float64)
8. flat_model (dtype: object)
9. lease_commence_date (dtype: int64)
10. resale_price (dtype: int64)

First 5 rows:
     month        town flat_type block       street_name storey_range  \
0  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
1  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     04 TO 06   
2  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
3  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     07 TO 09   
4  1990-01  ANG MO KIO    3 ROOM   216  ANG MO KIO AVE 1     04 TO 06   

   floor_area_sqm      flat_model  lease_commence_date  resale_price  
0            31.0        IMPROVED                 1977     

In [5]:
import pandas as pd
import numpy as np

file_names = [
    '../data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv', 
    '../data/Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv', 
    '../data/Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014.csv', 
    '../data/Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv', 
    '../data/Resale flat prices based on registration date from Jan-2017 onwards.csv'
]

initial_counts = []
final_counts = []
dfs = []

def parse_remaining_lease(lease_val):
    """
    Converts a lease value from a string like "86 years 1 months" to a numeric value (in years).
    Option 1 (default): returns years + (months/12).
    Option 2 (if you want to clamp months): simply return the years portion.
    """
    try:
        # If it's a string, split and parse
        if isinstance(lease_val, str):
            lease_val = lease_val.replace(',', '').strip()
            parts = lease_val.split()
            # parts[0] should be the number of years
            years = float(parts[0])
            # If there is a month component, process it:
            if len(parts) >= 3:
                months = float(parts[2])
                return years + months / 12.0
              
            else:
                return years
        else:
            return float(lease_val)
    except Exception as e:
        return np.nan

for file in file_names:
    df = pd.read_csv(file)
    
    initial_count = len(df)
    initial_counts.append(initial_count)
    
    df = df.dropna()
    cleaned_count = len(df)
    final_counts.append(cleaned_count)
    
    if 'remaining_lease' not in df.columns:
        # Compute remaining_lease for older files:
        df['year'] = df['month'].str[:4].astype(int)
        df['remaining_lease'] = (df['lease_commence_date'] + 99) - df['year']
        df.drop(columns=['year'], inplace=True)
    else:
        # Convert string formatted remaining_lease to numeric
        df['remaining_lease'] = df['remaining_lease'].apply(parse_remaining_lease)
    
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

total_initial = sum(initial_counts)
total_final = len(merged_df)
lost_rows = total_initial - total_final

print("Total initial rows across all tables:", total_initial)
print("Total rows after cleaning and computing remaining_lease:", total_final)
print("Total data points lost during cleaning:", lost_rows)

df.to_csv("merged_data.csv", index=False)



Total initial rows across all tables: 948962
Total rows after cleaning and computing remaining_lease: 948962
Total data points lost during cleaning: 0


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


df = pd.read_csv("merged_data.csv")


# These columns have few unique values so one-hot encoding is appropriate.
df = pd.get_dummies(df, columns=["town", "flat_type"], drop_first=True)

# The storey_range is a string like "10 TO 12". We extract the minimum and maximum floors,
# then compute their average.
df["min_floor"] = df["storey_range"].str[:2].astype(int)  # Extracts first two characters as min floor
df["max_floor"] = df["storey_range"].str[-2:].astype(int)  # Extracts last two characters as max floor
df["avg_floor"] = (df["min_floor"] + df["max_floor"]) / 2     # Computes the average floor
# Remove the original storey_range and the temporary min/max columns
df.drop(columns=["storey_range", "min_floor", "max_floor"], inplace=True)

# Target encoding replaces each category with the mean of the target variable (resale_price)
# for that category.
te = TargetEncoder(cols=["street_name", "block"])
# We transform these columns based on the overall resale_price means.
# (In a train/test setting, be sure to fit on the training set only to avoid data leakage.)
df[["street_name", "block"]] = te.fit_transform(df[["street_name", "block"]], df["resale_price"])

# Here we standardize floor_area_sqm, remaining_lease, and the computed avg_floor.
scaler = StandardScaler()
df[["floor_area_sqm", "remaining_lease", "avg_floor"]] = scaler.fit_transform(
    df[["floor_area_sqm", "remaining_lease", "avg_floor"]]
)

# The dataframe now has:
# - One-hot encoded columns for town and flat_type.
# - Target-encoded numeric values for street_name and block.
# - A numeric column (avg_floor) derived from storey_range.
# - Normalized continuous features.

print(df.head())
df.to_csv("processed_data.csv", index=False)


     month          block    street_name  floor_area_sqm      flat_model  \
0  2017-01  457047.747212  392807.671303       -2.202680        Improved   
1  2017-01  454462.888871  417415.335664       -1.245533  New Generation   
2  2017-01  440983.378641  388398.175737       -1.245533  New Generation   
3  2017-01  448187.441176  392807.671303       -1.203918  New Generation   
4  2017-01  481115.977011  388398.175737       -1.245533  New Generation   

   lease_commence_date  remaining_lease  resale_price  town_BEDOK  \
0                 1979        -0.928203      232000.0       False   
1                 1978        -0.981360      250000.0       False   
2                 1980        -0.851421      262000.0       False   
3                 1980        -0.875046      265000.0       False   
4                 1980        -0.851421      265000.0       False   

   town_BISHAN  ...  town_TOA PAYOH  town_WOODLANDS  town_YISHUN  \
0        False  ...           False           False        F

In [9]:
import pandas as pd

def analyze_numerical_data(df, title="Data Analysis"):
    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
    print(f"=== {title} ===")
    print("Data Shape:", df.shape)
    print("\nNumeric Columns:", numeric_cols)
    print("\nSummary Statistics for Numeric Columns:")
    print(df[numeric_cols].describe())
    print("\nCorrelation Matrix for Numeric Columns:")
    print(df[numeric_cols].corr())
    print("\nUnique Value Counts for Numeric Columns:")
    for col in numeric_cols:
        print(f"{col}: {df[col].nunique()} unique values")
    print("\n" + "="*80 + "\n")

# --- Analysis for Raw Merged Data ---
raw_df = pd.read_csv("merged_data.csv")
analyze_numerical_data(raw_df, "Raw Merged Data Analysis")

# --- Analysis for Processed Merged Data ---
proc_df = pd.read_csv("processed_merged_data.csv")
analyze_numerical_data(proc_df, "Processed Merged Data Analysis")


=== Raw Merged Data Analysis ===
Data Shape: (202759, 11)

Numeric Columns: ['floor_area_sqm', 'lease_commence_date', 'remaining_lease', 'resale_price']

Summary Statistics for Numeric Columns:
       floor_area_sqm  lease_commence_date  remaining_lease  resale_price
count   202759.000000        202759.000000    202759.000000  2.027590e+05
mean        96.929859          1996.245094        74.429543  5.119104e+05
std         24.029811            14.188199        14.109244  1.796821e+05
min         31.000000          1966.000000        40.083333  1.400000e+05
25%         82.000000          1985.000000        62.833333  3.800000e+05
50%         93.000000          1996.000000        74.250000  4.800000e+05
75%        112.000000          2011.000000        88.583333  6.138880e+05
max        366.700000          2021.000000        97.750000  1.600000e+06

Correlation Matrix for Numeric Columns:
                     floor_area_sqm  lease_commence_date  remaining_lease  \
floor_area_sqm        

FileNotFoundError: [Errno 2] No such file or directory: 'processed_merged_data.csv'